In [66]:
#bring in packages
import pandas as pd
import numpy as np
import datetime

import matplotlib # base python plotting library
%matplotlib inline  
import matplotlib.pyplot as plt # more plotting stuff 

# geocoding libraries
from geopy.geocoders import Nominatim

from time import sleep


Basic Cleaning

In [2]:
#read in data saved to hard drive
df = pd.read_csv(rf'~\Python_Projects\raw_data\City of Oakland Housing Habitability Complaints 2013 to 2018_raw.csv')

In [3]:
#drop columns without information
columns_to_drop = ['Project Name','Short Notes','Unnamed: 8']
df.drop(columns = columns_to_drop, inplace = True)

In [4]:
#make the description column lower case to improve later search function
df['Description'] = df['Description'].str.lower()

In [5]:
# need to understand if there are any duplicates in the record number
# df['Record Number'].nunique()
# there are 14789 records
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14789 entries, 0 to 14788
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Filed Date     14789 non-null  object
 1   Record Number  14789 non-null  object
 2   Record Type    14789 non-null  object
 3   Address        14788 non-null  object
 4   Description    14779 non-null  object
 5   Status         14782 non-null  object
dtypes: object(6)
memory usage: 693.4+ KB


In [6]:
#drop all rows that have a null description since that's our variable of interest
df.dropna(subset = ['Description'],inplace= True)
df.info()
#14779 records left

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14779 entries, 0 to 14788
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Filed Date     14779 non-null  object
 1   Record Number  14779 non-null  object
 2   Record Type    14779 non-null  object
 3   Address        14778 non-null  object
 4   Description    14779 non-null  object
 5   Status         14772 non-null  object
dtypes: object(6)
memory usage: 808.2+ KB


In [7]:
#drop the one record that doesn't have an address
df.dropna(subset = ['Address'],inplace= True)
df.info()
#14778 records

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14778 entries, 0 to 14788
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Filed Date     14778 non-null  object
 1   Record Number  14778 non-null  object
 2   Record Type    14778 non-null  object
 3   Address        14778 non-null  object
 4   Description    14778 non-null  object
 5   Status         14771 non-null  object
dtypes: object(6)
memory usage: 808.2+ KB


In [8]:
#turn the date into a python datetime format
df['Filed Date'] = pd.to_datetime(df['Filed Date'])

In [9]:
#create new column of just the year for ease of use
df['Year'] = pd.DatetimeIndex(df['Filed Date']).year

In [10]:
df.groupby('Year').count()
#reduced records over time, (Note from Alex: I think this is related to a loss of inspection staff,
# shouldn't interpret it as fewer issues)

,Filed Date,Record Number,Record Type,Address,Description,Status
Year,,,,,,
2013,4338,4338,4338,4338,4338,4338
2014,2367,2367,2367,2367,2367,2365
2015,1975,1975,1975,1975,1975,1974
2016,2016,2016,2016,2016,2016,2015
2017,2294,2294,2294,2294,2294,2291
2018,1788,1788,1788,1788,1788,1788


In [11]:
# dropping 2013 to match the 2014-2018 ACS 5 year sample data
df = df.loc[df['Year'] != 2013].copy()

Exploring the distributions of repeat violators

In [12]:
df['Address'].describe()
# there are 2628 records where addresses are repeated 

count                             10440
unique                             8190
top       344 13TH ST, Oakland CA 94612
freq                                 54
Name: Address, dtype: object

In [13]:
#trying to understand how those 2628 records are distributed 
# dups.df = 
duplicates = df.groupby('Address').count()
duplicates = duplicates[duplicates['Filed Date'] > 1]

In [14]:
duplicates
#1761 addresses have repeat violations

,Filed Date,Record Number,Record Type,Description,Status,Year
Address,,,,,,
"0 OUTLOOK AVE, Oakland CA 94608",2,2,2,2,2,2
"0 SHEPHERD CANYON RD, Oakland CA 94611",2,2,2,2,2,2
"1 LAKESIDE DR, Oakland CA 94612",3,3,3,3,3,3
"1 NORTH HILL CT, Oakland CA 94618",2,2,2,2,2,2
"100 9TH ST, Oakland CA 94607",2,2,2,2,2,2
...,...,...,...,...,...,...
"9928 HOLLY ST, Oakland CA 94603",2,2,2,2,2,2
"9939 INTERNATIONAL BLVD, Oakland CA 94603",5,5,5,5,5,5
"9943 VOLTAIRE AVE, Oakland CA 94603",3,3,3,3,3,3


In [15]:
duplicates.groupby('Filed Date').count()
#most duplicate addresses are 2-4 times, only a few have many complaints

,Record Number,Record Type,Description,Status,Year
Filed Date,,,,,
2,1039,1039,1039,1039,1039
3,249,249,249,249,249
4,73,73,73,73,73
5,34,34,34,34,34
6,14,14,14,14,14
7,10,10,10,10,10
8,1,1,1,1,1
9,1,1,1,1,1
10,2,2,2,2,2


Creating 'Healthy Housing Issue' Variable
- search the description variable for existence of any of a list of key words

In [16]:
# make variables if any of the following words show up in the string. mold, mildew, water, 
# leak, plumbing, moist, seepage, black, ventilation, flood, rot, condensation, corroded, and mildrew 
key_words = ['leak', 'plumbing', 'moist', 'seepage', 'black', 'ventilation', 'flood', 'rot', 'condensation',\
             'corroded', 'mildrew', 'mold', 'mildew', 'water', 'asbestos']

for i in range(len(key_words)): #loop through the key words to find if any of the words exist 
    df[key_words[i]] = df['Description'].str.find(key_words[i]) #create a new column for each word that
    #populates -1 if no, index if yes
    
for i in range(len(key_words)): #iterate through the length of the list of key words
    df[key_words[i]] = df[key_words[i]].apply(lambda x: 0 if x == -1 else 1) #apply a transformation to each column
    #to turn -1 into 0s and other numbers into 1s
        

In [17]:
#quick look at which words had the most hits
# need to figure this out, maybe learn the pivot table method? or a fancier groupbye?

In [44]:
#create new dataset that only keeps those records with a True in them.
#hh is shorthand for 'healthy housing'
df['HH_Complaint'] = 0
for i in range(len(key_words)):
    df['HH_Complaint'] += df[key_words[i]]
hh_df = df.loc[df['HH_Complaint'] > 0].copy(deep=True)
hh_df.columns

Index(['Filed Date', 'Record Number', 'Record Type', 'Address', 'Description',
       'Status', 'Year', 'leak', 'plumbing', 'moist', 'seepage', 'black',
       'ventilation', 'flood', 'rot', 'condensation', 'corroded', 'mildrew',
       'mold', 'mildew', 'water', 'asbestos', 'HH_Complaint'],
      dtype='object')

In [45]:
#then, cleaning up the new df to get rid of the individual columns
hh_df.drop(columns = key_words, inplace = True)
hh_df.columns    

Index(['Filed Date', 'Record Number', 'Record Type', 'Address', 'Description',
       'Status', 'Year', 'HH_Complaint'],
      dtype='object')

In [54]:
#appears that 2886 of the total 10440 are housing habitability complaints
print(df.shape)
print(hh_df.shape)

(10440, 23)
(2886, 5)


In [55]:
#condensing hh_df to only focus on each address, removing any duplicates
hh_df = hh_df.groupby('Address').sum().copy(deep=True)
hh_df.reset_index(inplace = True)

In [56]:
hh_df.head()

,Address,Year,HH_Complaint,latitude,longitude
0,"0 LINDEN ST, Oakland CA 94602",2014,1,37.825085,0.0
1,"0 THORNDALE DR, Oakland CA 94603",2015,1,0.000000,0.0
2,"100 9TH ST, 208, Oakland CA 94607",2018,1,0.000000,0.0
3,"100 9TH ST, Oakland CA 94607",4036,2,0.000000,0.0
4,"1000 43RD ST, UNIT 7, Oakland CA 94608",2017,3,0.000000,0.0


In [57]:
hh_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2886 entries, 0 to 2885
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Address       2886 non-null   object 
 1   Year          2886 non-null   int64  
 2   HH_Complaint  2886 non-null   int64  
 3   latitude      2886 non-null   float64
 4   longitude     2886 non-null   float64
dtypes: float64(2), int64(2), object(1)
memory usage: 112.9+ KB


Geocoding Data Frame to Turn Addresses into Lat/Long

In [74]:
# following this: https://stackoverflow.com/questions/31414481/new-column-with-coordinates-using-geopy-pandas
geolocator = Nominatim(user_agent="OaklandGeocoder")


In [85]:
#testing API locator on first record
location = geolocator.geocode('100 9TH ST, Oakland CA 94607')
type(location)

geopy.location.Location

In [76]:
#create blank columns for the lat and long
hh_df['latitude'] = np.nan
hh_df['longitude'] = np.nan

In [98]:
#the geopy code does not take kindly to having any unit/apt information in it and it breaks.
#need to remove the unit/apt part of the address.
#first, split the address before and after oakland

address_split = hh_df['Address'].str.split(', Oakland', expand=True)

In [104]:
#second, split the first half of the address along thet 
address_split_first = address_split[0].str.split(',', expand=True)

In [105]:
#then, join the first part of 'address_split_first' and the second part of 'address_split' with Oakland in the middle
hh_df['Address_recode'] = address_split_first[0] + ', Oakland' + address_split[1]

In [106]:
hh_df.head()

,Address,Year,HH_Complaint,latitude,longitude,first,second,Address_recode
0,"0 LINDEN ST, Oakland CA 94602",2014,1,37.825085,-122.278129,0,1,"0 LINDEN ST, Oakland CA 94602"
1,"0 THORNDALE DR, Oakland CA 94603",2015,1,37.844310,-122.203466,0,1,"0 THORNDALE DR, Oakland CA 94603"
2,"100 9TH ST, 208, Oakland CA 94607",2018,1,NaN,NaN,0,1,"100 9TH ST, Oakland CA 94607"
3,"100 9TH ST, Oakland CA 94607",4036,2,NaN,NaN,0,1,"100 9TH ST, Oakland CA 94607"
4,"1000 43RD ST, UNIT 7, Oakland CA 94608",2017,3,NaN,NaN,0,1,"1000 43RD ST, Oakland CA 94608"


In [107]:
#populating the lat and long for 2886. Code takes 48 minutes!!
for i in range(len(hh_df)): #loop over each row
    location = geolocator.geocode(hh_df.at[i, 'Address_recode']) #grab the geolocation of each row, assign to location
    hh_df.at[i, 'latitude'] = location.latitude #assign the latitude 
    hh_df.at[i, 'longitude'] = location.longitude #assign the longitude
    sleep(1) #insert one second pause so they don't lock you out per Nominatim's usage agreement

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=1110+CHESTER+ST%2C+Oakland+CA+94607&format=json&limit=1 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x000002352EBED160>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=1)'))

In [86]:
hh_df.head(10)

,Address,Year,HH_Complaint,latitude,longitude
0,"0 LINDEN ST, Oakland CA 94602",2014,1,37.825085,-122.278129
1,"0 THORNDALE DR, Oakland CA 94603",2015,1,37.844310,-122.203466
2,"100 9TH ST, 208, Oakland CA 94607",2018,1,NaN,NaN
3,"100 9TH ST, Oakland CA 94607",4036,2,NaN,NaN
4,"1000 43RD ST, UNIT 7, Oakland CA 94608",2017,3,NaN,NaN
5,"1000 E 33RD ST, Oakland CA 94610",2014,2,NaN,NaN
6,"1000 WALKER AVE, Oakland CA 94610",2015,2,NaN,NaN
7,"10000 MACARTHUR BLVD, Oakland CA 94605",4032,3,NaN,NaN
8,"10000 MACARTHUR BLVD, UNIT 104, Oakland CA 94605",2016,2,NaN,NaN
9,"10000 MACARTHUR BLVD, UNIT 14, Oakland CA 94605",2016,1,NaN,NaN


Turning DataFrame into Census Tract Level Data
- should include columns such as number of total complaints and number of healthy housing complaints
- should easily merge into the ACS census tract data

In [72]:
range(len(hh_df))

range(0, 2886)